In [23]:
%pip install pandas
%pip install langchain langchain-community sentence_transformers faiss-cpu langchain-huggingface
%pip install ipywidgets
%pip install datasets
#если есть возможность ставить видеокарту, вместо последнего пишем faiss-gpu
#Чтобы работало без сбоев, можно зафиксировать версии, например langchain == 0.2.6 sentence_transformers == 3.0.1 faiss-cpu == 1.7.4 langchain-community == 0.2.6

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-17.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
Using cached pyarrow-17.0.0-cp312-cp312-win_amd64.whl (25.1 MB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached pyarrow_hotfix-0.6-py3-non

In [2]:
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community. vectorstores import FAISS
from langchain.document_loaders import TextLoader
import re
import pandas as pd
from tqdm.autonotebook import tqdm, trange

C:\Users\volko\AppData\Local\Temp\ipykernel_27896\162743195.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Для создания embeddings используем rubert-tiny2. OpenAI embeddings лучше, но за деньги

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name='cointegrated/rubert-tiny2',
                                        # multi_process=True, model_kwargs={"device": "cuda"},
                                         encode_kwargs={"normalize_embeddings": True})
# есть работаем без виде карты удаляем или закомментируем кусок кода выше model_kwargs={"device": "cuda"}, иначе если не найдет видеокарту, выдаст ошибку. Без видеокарты будет работать медленнее

Будем работать с файлом 'https://opendata.trudvsem.ru/csv/cv.csv'. Там хранятся резюме

In [4]:
import requests

def download_file(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
        
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded successfully: {filename}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while downloading the file: {e}")

In [5]:
import os
url = 'https://opendata.trudvsem.ru/csv/cv.csv'
filename = './data/cv.csv'
if not os.path.isfile(filename):
    download_file(url, filename)
else:
    print(f'File {filename} already exists')


File ./data/cv.csv already exists


Преобразуем к dataframe

In [6]:
df_resum = pd.read_csv('./data/cv.csv', on_bad_lines='skip', nrows=100000, sep='|')

Смотрим на данные.

In [7]:
df_resum.head(3)

,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,dateCreate,...,softSkills,workExperienceList,scheduleType,salary,busyType,retrainingCapability,businessTrip,languageKnowledge,relocation,innerInfo
0,1825e760-d78a-11ee-9e37-c7bd3a8c3ec7,23dcd330-d789-11ee-b98a-5b073a2fbeea,6600000000000,6600001500000,"Свердловская-область,-г-Новоуральск",1990-08-24T16:00:00+0400,Женский,33.0,уборщик,2024-03-01T08:11:00+0300,...,[],"[{""companyName"": ""ООО ПАРКОВЫЙ РАЙОН-1"", ""date...",Полный-рабочий-день,34000,Полная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""c3a664a0-216a-11e5-91a0-1ff7059456..."
1,1825e440-ddbc-11ed-bd69-af80dfa0be97,ace533a0-e71e-11ec-9220-4796a92b3c9b,2300000000000,2300900001200,"Краснодарский-край,-Гулькевичский-район,-Гирей...",2006-09-25T00:00:00+0400,Мужской,17.0,рабочий,2023-04-18T10:39:04+0300,...,[],[],Неполный-рабочий-день,16242,Временная,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""cecb61c0-7d5c-11ea-88ba-5d9e90ab95..."
2,1825e3c0-b9a9-11eb-b4f1-c92c5fcfa4a0,428076c0-ac5e-11ea-9ed9-7bf9d8e248ac,6100000000000,6100000000000,Ростовская-область,1981-04-21T00:00:00+0400,Женский,43.0,личный-водитель,2021-05-20T23:22:27+0300,...,[],[],Полный-рабочий-день,45000,Полная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""8e0bc6b0-249d-11e5-b38b-1ff7059456..."


In [8]:
df_resum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           100000 non-null  object 
 1   candidateId                  100000 non-null  object 
 2   stateRegionCode              100000 non-null  int64  
 3   locality                     100000 non-null  int64  
 4   localityName                 99632 non-null   object 
 5   birthday                     97387 non-null   object 
 6   gender                       100000 non-null  object 
 7   age                          97387 non-null   float64
 8   positionName                 99993 non-null   object 
 9   dateCreate                   99999 non-null   object 
 10  dateModify                   100000 non-null  object 
 11  publishedDate                99740 non-null   object 
 12  academicDegree               0 non-null       float64
 13  

Чтобы узнать уникальные значения в колонке, используем команду

In [9]:
df_resum['localityName'].unique()

array(['Свердловская-область,-г-Новоуральск',
       'Краснодарский-край,-Гулькевичский-район,-Гирей-поселок-городского-типа',
       'Ростовская-область', ..., 'г-Севастополь,-Орлиное-село',
       'Воронежская-область,-Нижнедевицкий-район,-совхоза-"Кучугуровский"-поселок',
       'Башкортостан-республика,-Ишимбайский-район,-Канакаево-деревня'],
      dtype=object)

Выбираем только слоблцы, значимые для нашей задачи

In [10]:
#col = [
#	"localityName",
#	"birthday",
#	"age",
#	"gender",
#	"positionName",
#	"experience",
#	"educationList",
#	"hardSkills",
#	"softSkills",
#	"workExperienceList",
#	"companyName",
#	"dateFrom",
#	"dateTo",
#	"demands",
#	"jobTitle",
#	"relevant",
#	"tуре",
#	"scheduleType",
#	"salary",
#	"busyType",
#	"languageKnowledge",
#	"relocation"
#]

col=['localityName', 'birthday', 'age', 'gender', 'positionName',
     'experience',
      'educationList', 'hardSkills', 'softSkills', 'workExperienceList',
      'scheduleType', 'salary', 'busyType', 'languageKnowledge', 'relocation']

In [11]:
df_resum=df_resum[col]
df_resum.head(3)

,localityName,birthday,age,gender,positionName,experience,educationList,hardSkills,softSkills,workExperienceList,scheduleType,salary,busyType,languageKnowledge,relocation
0,"Свердловская-область,-г-Новоуральск",1990-08-24T16:00:00+0400,33.0,Женский,уборщик,3,"[{""graduateYear"": 2006, ""instituteName"": ""СОШ ...",[],[],"[{""companyName"": ""ООО ПАРКОВЫЙ РАЙОН-1"", ""date...",Полный-рабочий-день,34000,Полная-занятость,[],Не-готов-к-переезду
1,"Краснодарский-край,-Гулькевичский-район,-Гирей...",2006-09-25T00:00:00+0400,17.0,Мужской,рабочий,0,"[{""type"": ""Education""}]",[],[],[],Неполный-рабочий-день,16242,Временная,[],Не-готов-к-переезду
2,Ростовская-область,1981-04-21T00:00:00+0400,43.0,Женский,личный-водитель,0,[],[],[],[],Полный-рабочий-день,45000,Полная-занятость,[],Не-готов-к-переезду


In [12]:
df_resum.columns = ['НазваниеНаселенногоПункта','ДатаРождения','Возраст','Пол','Должность',
     'ОпытРаботы',
     'ИнформацияОбразовании','ПрофессиональныеНавыки','ГибкиеНавыки','ИнформацияОпытРаботы',
     'ГрафикРаботы','ЗП','ТипЗанятости','УровниВладенияЯзыками','ГотовностьКПереезду']

In [15]:
df_resum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   НазваниеНаселенногоПункта  99632 non-null   object 
 1   ДатаРождения               97387 non-null   object 
 2   Возраст                    97387 non-null   float64
 3   Пол                        100000 non-null  object 
 4   Должность                  99993 non-null   object 
 5   ОпытРаботы                 100000 non-null  int64  
 6   ИнформацияОбразовании      100000 non-null  object 
 7   ПрофессиональныеНавыки     100000 non-null  object 
 8   ГибкиеНавыки               100000 non-null  object 
 9   ИнформацияОпытРаботы       100000 non-null  object 
 10  ГрафикРаботы               100000 non-null  object 
 11  ЗП                         100000 non-null  int64  
 12  ТипЗанятости               89979 non-null   object 
 13  УровниВладенияЯзыками      100

In [18]:
df_resum.head(3)

,НазваниеНаселенногоПункта,ДатаРождения,Возраст,Пол,Должность,ОпытРаботы,ИнформацияОбразовании,ПрофессиональныеНавыки,ГибкиеНавыки,ИнформацияОпытРаботы,ГрафикРаботы,ЗП,ТипЗанятости,УровниВладенияЯзыками,ГотовностьКПереезду
0,"Свердловская-область,-г-Новоуральск",1990-08-24T16:00:00+0400,33.0,Женский,уборщик,3,"[{""graduateYear"": 2006, ""instituteName"": ""СОШ ...",[],[],"[{""companyName"": ""ООО ПАРКОВЫЙ РАЙОН-1"", ""date...",Полный-рабочий-день,34000,Полная-занятость,[],Не-готов-к-переезду
1,"Краснодарский-край,-Гулькевичский-район,-Гирей...",2006-09-25T00:00:00+0400,17.0,Мужской,рабочий,0,"[{""type"": ""Education""}]",[],[],[],Неполный-рабочий-день,16242,Временная,[],Не-готов-к-переезду
2,Ростовская-область,1981-04-21T00:00:00+0400,43.0,Женский,личный-водитель,0,[],[],[],[],Полный-рабочий-день,45000,Полная-занятость,[],Не-готов-к-переезду


In [13]:
#col = [
#    "НазваниеНаселенногоПункта",
#    "ДатаРождения",
#    "Возраст",
#    "Пол",
#    "Должность",
#    "ОпытРаботы",
#    "ИнформацияОбразовании",
#    "ПрофессиональныеНавыки",
#    "ГибкиеНавыки",
#    "ИнформацияОпытРаботы",
#    "НазваниеКомпании",
#    "ДатаНачала",
#    "ДатаОкончания",
#    "Обязанности",
#    "ДолжностьНаРаботе",
#    "Релевантность",
#    "Тип",
#    "ГрафикРаботы",
#    "Зарплата",
#    "ТипЗанятости",
#    "УровеньЗнанияЯзыков",
#    "ГотовностьКПереезду"
#]


In [38]:
magic_word = "qqqrrrqqq"

Переводим всё в текстовый вид. iterrows выдаёт название индекса и содержание самого ряда. Использование абракадабры — авторский приём для сохранения индекса. С её помощью мы избегаем путаницы с другими цифрами, которые могут появиться в тексте.
Надо проверить в документации. Возможно можно использовать сразу csv. Не очень понятно, зачем в текст переводить

In [19]:
# Сбор из таблицы текстовый файл, если данные изначально в таблице
output_path = './data/cvtxt.txt'
output_text = ""

idx = 1
# Перебор строк DataFrame
for index, row in df_resum.iterrows():
    if idx % 1000 == 0:
        with open(output_path, 'a', encoding='utf-8') as file:
            file.write(output_text)
        output_text = ""
        print(f"{idx} records processed")

    row_text = ""
    # Мы через "абракадабру" .qqqrrrqqq\n сохраняем связку с номером
    row_text+=f'\n{index}.{magic_word}\n'
    # Проходим по каждой колонка и плюсуем её к тексту резюме с новой строки \n
    for col in df_resum.columns:
        row_text += f"{col}: {row[col]}\n"
    # В конце добавляем ещё одну "абракадабру" чтобы было понятно что резюме закончилось
    row_text += "\nENDOFR\n\n"
    output_text += row_text
    idx = idx + 1

# Запись результата в файл
#with open(output_path, 'w') as file:
#    file.write(output_text)

print("Текст успешно записан в файл")

1000 records processed
2000 records processed
3000 records processed
4000 records processed
5000 records processed
6000 records processed
7000 records processed
8000 records processed
9000 records processed
10000 records processed
11000 records processed
12000 records processed
13000 records processed
14000 records processed
15000 records processed
16000 records processed
17000 records processed
18000 records processed
19000 records processed
20000 records processed
21000 records processed
22000 records processed
23000 records processed
24000 records processed
25000 records processed
26000 records processed
27000 records processed
28000 records processed
29000 records processed
30000 records processed
31000 records processed
32000 records processed
33000 records processed
34000 records processed
35000 records processed
36000 records processed
37000 records processed
38000 records processed
39000 records processed
40000 records processed
41000 records processed
42000 records processed
4

In [72]:
def read_resume_file(file_path: str) -> list[str]:
    records = []
    current_record = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.endswith('.qqqrrrqqq'):
                if current_record:
                    records.append('\n'.join(current_record))
                current_record = [line]
            elif line == 'ENDOFR':
                current_record.append(line)
                records.append('\n'.join(current_record))
                current_record = []
            elif current_record:
                current_record.append(line)

    # Handle case where file doesn't end with ENDOFR
    if current_record:
        records.append('\n'.join(current_record))

    return records

In [73]:
source_chunks = read_resume_file(output_path)

In [76]:
print(source_chunks[0])

0.qqqrrrqqq
НазваниеНаселенногоПункта: Свердловская-область,-г-Новоуральск
ДатаРождения: 1990-08-24T16:00:00+0400
Возраст: 33.0
Пол: Женский
Должность: уборщик
ОпытРаботы: 3
ИнформацияОбразовании: [{"graduateYear": 2006, "instituteName": "СОШ №59 г.Новоуральска", "type": "Education"}]
ПрофессиональныеНавыки: []
ГибкиеНавыки: []
ИнформацияОпытРаботы: [{"companyName": "ООО ПАРКОВЫЙ РАЙОН-1", "dateFrom": "2020-10-15T00:00:00+0300", "dateTo": "2024-02-15T00:00:00+0300", "demands": "<p>-</p>", "jobTitle": "УБОРЩИК", "relevant": false, "type": "WorkExperience"}]
ГрафикРаботы: Полный-рабочий-день
ЗП: 34000
ТипЗанятости: Полная-занятость
УровниВладенияЯзыками: []
ГотовностьКПереезду: Не-готов-к-переезду

ENDOFR


In [ ]:
# Преобразоание строк в объекты Document (lang_chain)
documents = [Document (page_content=chunk) for chunk in source_chunks]

# Создание базы знаний с использованием FAISS и ранее заданного эмбеддинга
db = FAISS.from_documents(documents, embedding_model)
print (f'База знаний создана!')

In [ ]:
with open('./data/vacancy.txt', 'r', encoding='utf-8') as file:
    vacancy = file.read()


In [ ]:
k = 5
cvs = db.similarity_search (vacancy, k=k)
for cv in cvs:
    print(cv.page_content)


In [40]:
from datasets import Dataset

cv_dataset = Dataset.from_pandas(df_resum)

from typing import Dict, Any

def data2text(row: Dict[str, Any], idx: int) -> Dict[str, str]:
    # Start with the row index and the specified prefix
    text = f"\n{idx}.{magic_word}\n"
    
    for column, value in row.items():
        if value is not None:  # Check if the value is not None
            text += f"{column}: {value}\n"
    
    # Add the end marker
    text += "\nENDOFR\n\n"
    
    return {"text": text}

# Apply the function to the dataset
cv_dataset = cv_dataset.map(data2text, with_indices=True)

cv_dataset["text"][0]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

'\n0.qqqrrrqqq\nНазваниеНаселенногоПункта: Свердловская-область,-г-Новоуральск\nДатаРождения: 1990-08-24T16:00:00+0400\nВозраст: 33.0\nПол: Женский\nДолжность: уборщик\nОпытРаботы: 3\nИнформацияОбразовании: [{"graduateYear": 2006, "instituteName": "СОШ №59 г.Новоуральска", "type": "Education"}]\nПрофессиональныеНавыки: []\nГибкиеНавыки: []\nИнформацияОпытРаботы: [{"companyName": "ООО ПАРКОВЫЙ РАЙОН-1", "dateFrom": "2020-10-15T00:00:00+0300", "dateTo": "2024-02-15T00:00:00+0300", "demands": "<p>-</p>", "jobTitle": "УБОРЩИК", "relevant": false, "type": "WorkExperience"}]\nГрафикРаботы: Полный-рабочий-день\nЗП: 34000\nТипЗанятости: Полная-занятость\nУровниВладенияЯзыками: []\nГотовностьКПереезду: Не-готов-к-переезду\n\nENDOFR\n\n'

In [42]:
from transformers import AutoTokenizer, AutoModel
model_ckpt='cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)



In [44]:
import torch

device = torch.device("cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

As we mentioned earlier, we’d like to represent each entry in our GitHub issues corpus as a single vector, so we need to “pool” or average our token embeddings in some way. One popular approach is to perform CLS pooling on our model’s outputs, where we simply collect the last hidden state for the special [CLS] token. The following function does the trick for us:

In [43]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

Next, we’ll create a helper function that will tokenize a list of documents, place the tensors on the GPU, feed them to the model, and finally apply CLS pooling to the outputs:

In [45]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [46]:
embedding = get_embeddings(cv_dataset["text"][0])
embedding.shape

torch.Size([1, 312])

In [47]:
embeddings_dataset = cv_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [48]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/100 [00:00<?, ?it/s]

Dataset({
    features: ['НазваниеНаселенногоПункта', 'ДатаРождения', 'Возраст', 'Пол', 'Должность', 'ОпытРаботы', 'ИнформацияОбразовании', 'ПрофессиональныеНавыки', 'ГибкиеНавыки', 'ИнформацияОпытРаботы', 'ГрафикРаботы', 'ЗП', 'ТипЗанятости', 'УровниВладенияЯзыками', 'ГотовностьКПереезду', 'text', 'embeddings'],
    num_rows: 100000
})

In [60]:
question = "Должность: Оператор. Зарплата не выше 25 тыс. Опыт работы более одного года"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 312)

In [61]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [62]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [54]:
import pandas as pd

def print_dataframe_rows(df):
    for index, row in df.iterrows():
        print("=" * 32)
        print(f"RowID: {index}")
        for column, value in row.items():
            print(f"{column}: {value}")
        print("=" * 32)
        print()  # Add an empty line between rows for better readability

In [63]:
print_dataframe_rows(samples_df)

RowID: 4
НазваниеНаселенногоПункта: Ленинградская-область,-Всеволожский-район,-г.-Мурино
ДатаРождения: 1990-05-24T00:00:00+0400
Возраст: 34.0
Пол: Женский
Должность: помощник-руководителя
ОпытРаботы: 9
ИнформацияОбразовании: [{"faculty": "Менеджмента", "graduateYear": 2013, "instituteName": "Санкт-Петербургский Государственный университет Промышленных Технологий и Дизайна", "qualification": "Менеджер", "speciality": "Менеджмент", "type": "Education"}]
ПрофессиональныеНавыки: []
ГибкиеНавыки: []
ИнформацияОпытРаботы: [{"achievements": "<p>Пришла в компанию на должность менджер в оптовый отдел. Перешла в розниные салоны, добилась новой должности. Ранее такой должности не было. Считаю своим достижением работу в одной компании на протяжении почти 9 лет. </p>", "companyName": "ООО \"БТ-Дизайн\"", "dateFrom": "2013-10-01T01:00:00+0400", "dateTo": "2022-08-01T00:00:00+0300", "demands": "<p>Консультация клиентов по имеющемуся ассортимету. Выставление счетов, ТТН, счет-фактур. Оформление довере